In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import json


In [2]:

class BaziCalculator:
    def __init__(self, headless=True):
        chrome_options = Options()
        if headless:
            chrome_options.add_argument("--headless")
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        
        self.driver = webdriver.Chrome(options=chrome_options)
        self.wait = WebDriverWait(self.driver, 10)
    
    def open_calculator(self):
        """Открывает страницу калькулятора"""
        url = "http://fengshui.su/v6/index.php?action=bazicalc&country=Россия&town=Москва"
        self.driver.get(url)
        time.sleep(2)
    
    def fill_form(self, data):
        """Заполняет форму данными"""
        try:
            # Имя
            if 'name' in data:
                name_field = self.wait.until(
                    EC.element_to_be_clickable((By.ID, "sname"))
                )
                name_field.clear()
                name_field.send_keys(data['name'])
            
            # Пол
            if 'gender' in data:
                if data['gender'].lower() == 'мужчина' or data['gender'].lower() == 'male':
                    gender_radio = self.driver.find_element(By.ID, "man")
                else:
                    gender_radio = self.driver.find_element(By.ID, "woman")
                gender_radio.click()
            
            # Дата рождения
            if 'birth_date' in data:
                # Пробуем разные форматы даты
                date_field = self.wait.until(
                    EC.element_to_be_clickable((By.ID, "date1"))
                )
                date_field.clear()
                date_field.send_keys(data['birth_date'])
            
            # Время рождения
            if 'birth_time' in data:
                time_field = self.driver.find_element(By.ID, "time")
                time_field.clear()
                time_field.send_keys(data['birth_time'])
            
            # Страна
            if 'country' in data:
                country_field = self.driver.find_element(By.ID, "countryinput")
                country_field.clear()
                country_field.send_keys(data['country'])
                time.sleep(1)  # Ждем обновления списка городов
            
            # Город
            if 'city' in data:
                city_field = self.driver.find_element(By.ID, "towninput")
                city_field.clear()
                city_field.send_keys(data['city'])
                time.sleep(1)
            
            # Широта и долгота (обновятся автоматически после выбора города)
            if 'latitude' in data and 'longitude' in data:
                lat_field = self.driver.find_element(By.ID, "latitude")
                lon_field = self.driver.find_element(By.ID, "longitude")
                lat_field.clear()
                lon_field.clear()
                lat_field.send_keys(str(data['latitude']))
                lon_field.send_keys(str(data['longitude']))
            
            # Часовой пояс
            if 'timezone' in data:
                tz_field = self.driver.find_element(By.ID, "gmt")
                tz_field.clear()
                tz_field.send_keys(str(data['timezone']))
            
            time.sleep(2)  # Даем время для применения изменений
            return True
            
        except Exception as e:
            print(f"Ошибка при заполнении формы: {e}")
            return False
    
    def calculate(self, calculation_type="card"):
        """Выполняет расчет"""
        try:
            # Выбираем тип расчета
            if calculation_type == "card":
                button = self.driver.find_element(By.ID, "buttonCalc")
            elif calculation_type == "hbday":
                button = self.driver.find_element(By.ID, "buttonEvents")
            elif calculation_type == "hbyear":
                button = self.driver.find_element(By.ID, "buttonHourBranch")
            elif calculation_type == "sunhex":
                button = self.driver.find_element(By.ID, "buttonHexagramm")
            else:
                button = self.driver.find_element(By.ID, "buttonCalc")
            
            button.click()
            
            # Ждем загрузки результатов
            time.sleep(3)
            
            # Получаем URL результата
            current_url = self.driver.current_url
            return current_url
            
        except Exception as e:
            print(f"Ошибка при расчете: {e}")
            return None
    
    def get_result(self):
        """Получает результат расчета (нужно адаптировать под конкретную страницу результата)"""
        try:
            # Здесь нужно адаптировать под структуру страницы с результатом
            # Это пример - вам нужно изучить HTML страницы результата
            
            result_data = {}
            
            # Пример извлечения данных (замените на реальные селекторы)
            try:
                # Ищем основные элементы результата
                elements = self.driver.find_elements(By.CLASS_NAME, "result-element")
                for element in elements:
                    # Адаптируйте логику извлечения данных
                    pass
                    
            except Exception as e:
                print(f"Не удалось извлечь данные: {e}")
            
            return result_data
            
        except Exception as e:
            print(f"Ошибка при получении результата: {e}")
            return {}
    
    def close(self):
        """Закрывает браузер"""
        self.driver.quit()

# Пример использования
def calculate_bazi(data):
    calculator = BaziCalculator(headless=False)  # False для отладки
    
    try:
        # Открываем калькулятор
        calculator.open_calculator()
        
        # Заполняем форму
        if calculator.fill_form(data):
            print("Форма успешно заполнена")
            
            # Выполняем расчет
            result_url = calculator.calculate("card")
            print(f"Расчет выполнен. URL: {result_url}")
            
            # Получаем результат
            result = calculator.get_result()
            
            # Сохраняем в словарь
            final_result = {
                'input_data': data,
                'result_url': result_url,
                'calculation_data': result
            }
            
            return final_result
        else:
            print("Ошибка при заполнении формы")
            return None
            
    except Exception as e:
        print(f"Общая ошибка: {e}")
        return None
    finally:
        calculator.close()


In [3]:

# Пример данных для расчета
sample_data = {
    'name': 'Максим Богданов',
    'gender': 'Мужчина',
    'birth_date': '1983-12-09',  # Формат: ГГГГ-ММ-ДД
    'birth_time': '14:30',
    'country': 'Россия',
    'city': 'Бор',
    # 'latitude': '55.7558',  # Автоматически определяется
    # 'longitude': '37.6173', # Автоматически определяется
    'timezone': '3'
}


In [4]:

# Выполняем расчет
if __name__ == "__main__":
    result = calculate_bazi(sample_data)
    
    if result:
        print("Результат расчета:")
        print(json.dumps(result, indent=2, ensure_ascii=False))
        
        # Сохраняем в файл
        with open('bazi_result.json', 'w', encoding='utf-8') as f:
            json.dump(result, f, indent=2, ensure_ascii=False)
        print("Результат сохранен в bazi_result.json")
    else:
        print("Не удалось выполнить расчет")

Форма успешно заполнена
Расчет выполнен. URL: http://fengshui.su/v6/application/views/bazicalc/bcalc.php?sname=%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%20%D0%91%D0%BE%D0%B3%D0%B4%D0%B0%D0%BD%D0%BE%D0%B2&town=%D0%91%D0%BE%D1%80&country=%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F&date=1209-08-19&time=14:30&notime=0&man=1&calendar=0&sun=0&color=1&gmt=3&latitude=55.75&longitude=37.61639&dtype=0&ectime=1&what=card
Результат расчета:
{
  "input_data": {
    "name": "Максим Богданов",
    "gender": "Мужчина",
    "birth_date": "1983-12-09",
    "birth_time": "14:30",
    "country": "Россия",
    "city": "Бор",
    "timezone": "3"
  },
  "result_url": "http://fengshui.su/v6/application/views/bazicalc/bcalc.php?sname=%D0%9C%D0%B0%D0%BA%D1%81%D0%B8%D0%BC%20%D0%91%D0%BE%D0%B3%D0%B4%D0%B0%D0%BD%D0%BE%D0%B2&town=%D0%91%D0%BE%D1%80&country=%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D1%8F&date=1209-08-19&time=14:30&notime=0&man=1&calendar=0&sun=0&color=1&gmt=3&latitude=55.75&longitude=37.61639&dtype=0&ectime=1&what=card",


In [ ]:
%pip install LunarCalendar

  Using cached LunarCalendar-0.0.9-py2.py3-none-any.whl (18 kB)
  Using cached ephem-4.2-cp39-cp39-win_amd64.whl (1.4 MB)


You should consider upgrading via the 'C:\Users\anast\Desktop\SF\IDE\Django_PostgreSQL\hronopunktura\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
import datetime
from lunarcalendar import Converter, Solar, Lunar, DateNotExist

date = datetime.datetime.now().date()
year = date.year
month = date.month
day = date.day

solar = Solar(year, month, day)
print(solar)
lunar = Converter.Solar2Lunar(solar)
print(lunar)
solar = Converter.Lunar2Solar(lunar)
print(solar)
print(solar.to_date(), type(solar.to_date()))

Solar(year=2025, month=9, day=29)
Lunar(year=2025, month=8, day=8, isleap=False)
Solar(year=2025, month=9, day=29)
2025-09-29 <class 'datetime.date'>


In [15]:
print(lunar)
print(lunar.from_date(date))

Lunar(year=2025, month=8, day=8, isleap=False)
Lunar(year=2025, month=8, day=8, isleap=False)
